# 1

In [1]:
import sqlite3


conn = sqlite3.connect("../data/asc.db")

conn.executescript("""
CREATE TABLE IF NOT EXISTS Student (
    stud_number INTEGER,
    first_name TEXT,
    last_name TEXT,
    gender TEXT,
    PRIMARY KEY("stud_number")
);
CREATE TABLE IF NOT EXISTS EmailAddress (
    email TEXT,
    stud_number INTEGER,
    PRIMARY KEY("email"),
    FOREIGN KEY("stud_number") REFERENCES "Student"("stud_numer")
);
CREATE TABLE IF NOT EXISTS association (
    ASSO_NAME TEXT,
    asso_desc TEXT,
    PRIMARY KEY("ASSO_NAME")
);
CREATE TABLE IF NOT EXISTS membership (
    STUD_NUMBER INTEGER,
    ASSO_NAME TEXT,
    stud_role TEXT,
    PRIMARY KEY("STUD_NUMBER"),
    FOREIGN KEY("stud_number") references "student"("stud_number"),
    FOREIGN KEY("asso_name") references "association"("asso_name")
);
CREATE TABLE IF NOT EXISTS ascEdition (
    YEAR INTEGER,
    registration_fee TEXT,
    PRIMARY KEY("YEAR")
);
CREATE TABLE IF NOT EXISTS registration (
    STUD_NUMBER INTEGER,
    YEAR INTEGER,
    registrationdate TEXT,
    payment_date TEXT,
    PRIMARY KEY("STUD_NUMBER", "YEAR"),
    FOREIGN KEY("stud_number") references "student"("stud_number"),
    FOREIGN KEY("year") references "ascEdition"("year")
);
CREATE TABLE IF NOT EXISTS login (
    USERNAME TEXT,
    password TEXT,
    PRIMARY KEY("USERNAME")
);
""")

# 2

In [2]:
import pandas as pd


memberships = pd.read_csv("../data/student_memberships.csv", sep=";")
registrations = pd.read_csv("../data/student_registrations.csv", sep=";")

# 3

In [3]:
display(memberships.head())
display(registrations.head())

,stud_number,first_name,last_name,gender,email,stud_role,asso_name,asso_desc
0,9699114,Ludwig,JACQUET,M,ludwig.jacquet@centralesupelec.fr,president,Band'à Joe,"la fanfare, où tout le monde débute un instrum..."
1,1978626,Rolande,ROUAULT,F,rolande.rouault@centralesupelec.fr,vice-president,Band'à Joe,"la fanfare, où tout le monde débute un instrum..."
2,5421882,Pierre Louis,PRIEUR,H,pierre louis.prieur@centralesupelec.fr,secretary,Band'à Joe,"la fanfare, où tout le monde débute un instrum..."
3,8737405,Jean-Brice,GENDREAU,garçon,jean-brice.gendreau@centralesupelec.fr,treasurer,Band'à Joe,"la fanfare, où tout le monde débute un instrum..."
4,9662058,Philppe,VEGER,garçon,philppe.veger@centralesupelec.fr,president,Barco,installation et exploitation de materiel video


,stud_number,first_name,last_name,gender,email,registration_date,payment_date,registration_fee,year
0,7719175,Eliane,CHOISNE,M,eliane.choisne@centralesupelec.fr,2015-10-08,2015-11-07,20.5,2016
1,7208231,Florian,COEFFARD,M,florian.coeffard@centralesupelec.fr,2015-10-01,2015-10-01,20.5,2016
2,5876984,Jacob,ROUSSIERE,garçon,jacob.roussiere@centralesupelec.fr,2015-10-10,2015-11-09,20.5,2016
3,2486399,Hortense,GUEFFIER,W,hortense.gueffier@centralesupelec.fr,2015-10-10,2015-10-10,20.5,2016
4,7874799,Timothée,COUILLAUD,H,timothee.couillaud@centralesupelec.fr,2015-09-27,2015-10-27,20.5,2016


In [4]:
db_student = registrations[["stud_number", "first_name", "last_name", "gender"]].drop_duplicates()
db_email_address = registrations[["stud_number", "email"]].drop_duplicates()
db_association = memberships[["asso_name", "asso_desc"]].drop_duplicates()
db_membership = memberships[["stud_number", "asso_name", "stud_role"]]
db_asc_edition = registrations[["registration_fee", "year"]].drop_duplicates()
db_registration = registrations[["stud_number", "year", "registration_date", "payment_date"]]
db_login = pd.DataFrame(columns=["username", "password"])

# 4

In [5]:
db_student = db_student.drop_duplicates()
db_email_address = db_email_address.drop_duplicates()
db_association = db_association.drop_duplicates()
db_membership = db_membership.drop_duplicates()
db_asc_edition = db_asc_edition.drop_duplicates()
db_registration = db_registration.drop_duplicates()
db_login = db_login.drop_duplicates()

# 5

In [6]:
assert len(db_student) == 602
assert len(db_association) == 11
assert len(db_email_address) == 602
assert len(db_asc_edition) == 3
assert len(db_registration) == 900
assert len(db_membership) == 895

# 6

In [7]:
from unidecode import unidecode


def normalize(gender: str) -> str:
    if unidecode(gender.lower()) in ["m", "h", "garcon"]:
        return "M"
    return "F"

db_student["gender"] = db_student["gender"].apply(normalize)

db_student["gender"].unique()

array(['M', 'F'], dtype=object)

# 7

In [8]:
assert set(db_student["gender"].unique()) == {"F", "M"}

# 8

In [9]:
db_registration["registration_date"] = pd.to_datetime(db_registration["registration_date"]).dt.strftime("%d-%m-%Y")
db_registration["payment_date"] = pd.to_datetime(db_registration["payment_date"]).dt.strftime("%d-%m-%Y")

db_registration

,stud_number,year,registration_date,payment_date
0,7719175,2016,08-10-2015,07-11-2015
1,7208231,2016,01-10-2015,01-10-2015
2,5876984,2016,10-10-2015,09-11-2015
3,2486399,2016,10-10-2015,10-10-2015
4,7874799,2016,27-09-2015,27-10-2015
...,...,...,...,...
895,8851393,2018,18-10-2017,18-10-2017
896,8446594,2018,17-09-2017,17-09-2017
897,549362,2018,24-09-2017,24-10-2017
898,8347453,2018,19-10-2017,19-10-2017


# 9

In [10]:
db_student["stud_number"]

0      7719175
1      7208231
2      5876984
3      2486399
4      7874799
        ...   
875    4456631
876    7099660
880    1642443
881    7456381
893    4184809
Name: stud_number, Length: 602, dtype: int64

In [11]:
db_student.to_sql("student", conn, if_exists="replace")
db_email_address.to_sql("emailAddress", conn, if_exists="replace")
db_association.to_sql("association", conn, if_exists="replace")
db_membership.to_sql("membership", conn, if_exists="replace")
db_asc_edition.to_sql("ascEdition", conn, if_exists="replace")
db_registration.to_sql("registration", conn, if_exists="replace")
db_login.to_sql("login", conn, if_exists="replace")

conn.commit()